<a href="https://colab.research.google.com/github/danvdb24/proust_transformer/blob/main/Proust-finetuning-Bloom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.4 MB/s 
     |████████████████████████████████| 5.8 MB 50.9 MB/s 
     |████████████████████████████████| 212 kB 75.2 MB/s 
     |████████████████████████████████| 132 kB 87.1 MB/s 
     |████████████████████████████████| 182 kB 75.5 MB/s 
     |████████████████████████████████| 127 kB 77.1 MB/s 
     |████████████████████████████████| 7.6 MB 49.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": 'train_dataset.txt', "validation": 'test_dataset.txt'})

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-77ff58e9c061f89f/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
datasets["train"][3]

{'text': 'Quelquefois, comme Ève naquit d’une côte d’Adam, une femme naissait pendant mon sommeil d’une fausse position de ma cuisse. Formée du plaisir que j’étais sur le point de goûter, je m’imaginais que c’était elle qui me l’offrait. Mon corps qui sentait dans le sien ma propre chaleur voulait s’y rejoindre, je m’éveillais. Le reste des humains m’apparaissait comme bien lointain auprès de cette femme que j’avais quittée, il y avait quelques moments à peine ; ma joue était chaude encore de son baiser, mon corps courbaturé par le poids de sa taille. Si, comme il arrivait quelquefois, elle avait les traits d’une femme que j’avais connue dans la vie, j’allais me donner tout entier à ce but : la retrouver, comme ceux qui partent en voyage pour voir de leurs yeux une cité désirée et s’imaginent qu’on peut goûter dans une réalité le charme du songe. Peu à peu son souvenir s’évanouissait, j’avais oublié la fille de mon rêve.'}

In [4]:
model_checkpoint = "bigscience/bloom-560m"

In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [7]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 568
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 17
    })
})

In [9]:
# block_size = tokenizer.model_max_length
block_size = 128

In [10]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' : une église, un quatuor, la rivalité de François Ier et de Charles-Quint. Cette croyance survivait pendant quelques secondes à mon réveil ; elle ne choquait pas ma raison, mais pesait comme des écailles sur mes yeux et les empêchait de se rendre compte que le bougeoir n’était plus allumé. Puis elle commençait à me devenir inintelligible, comme après la métempsycose les pensées d’une existence antérieure ; le sujet du livre se détachait de moi, j’étais libre de m’y appliquer ou non ; aussitôt je recouvrais la vue et j’étais bien étonné de trouver autour de'

In [13]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [14]:
from transformers import Trainer, TrainingArguments

In [15]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-proust",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=5.0
)

In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)

In [18]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 890
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 560
  Number of trainable parameters = 559214592


Epoch,Training Loss,Validation Loss
1,No log,3.025881
2,No log,2.309804
3,No log,1.652177
4,No log,1.121501
5,2.275400,0.938434


***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
Saving model checkpoint to bloom-560m-finetuned-proust/checkpoint-500
Configuration saved in bloom-560m-finetuned-proust/checkpoint-500/config.json
Model weights saved in bloom-560m-finetuned-proust/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=560, training_loss=2.1039096968514577, metrics={'train_runtime': 766.1832, 'train_samples_per_second': 5.808, 'train_steps_per_second': 0.731, 'total_flos': 1033186521907200.0, 'train_loss': 2.1039096968514577, 'epoch': 5.0})

In [19]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 37
  Batch size = 8


Perplexity: 2.56


In [20]:
trainer.save_model("finetuned-proust")

Saving model checkpoint to finetuned-proust
Configuration saved in finetuned-proust/config.json
Model weights saved in finetuned-proust/pytorch_model.bin


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch.nn import functional as F

# Load GPT-2 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained('finetuned-proust')

# Tokenize input phrase
phrase = 'Emmanuel et Nissim'

for i in range(100):
  inputs = tokenizer.encode(phrase, return_tensors='pt')

  # Get logits from last layer
  last_layer_logits = model(inputs).logits[:, -1, :]

  # Keep top 30 logits at max; stop if cumulative probability >= 1.0.
  top_logits = top_k_top_p_filtering(last_layer_logits, top_k=30, top_p=1.0)

  # Softmax the logits into probabilities
  probabilities = F.softmax(top_logits, dim=-1)

  # Generate next token
  generated_next_token = torch.multinomial(probabilities, num_samples=1)
  generated = torch.cat([inputs, generated_next_token], dim=-1)

  # Get result
  result_string = tokenizer.decode(generated.tolist()[0])

  # Print string
  # print(result_string)

  phrase = result_string

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/afe2e6f33eb135d254df849c74bb83322b53641c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/afe2e6f33eb135d254df849c74bb83322b53641c/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/afe2e6f33eb135d254df849c74bb83322b53641c/tokenizer_config.json
loading configuration file finetuned-proust/config.json
Model config BloomConfig {
  "_name_or_path": "finetuned-proust",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "i

In [24]:
print(result_string)

Emmanuel et Nissim, il leur avait donné, comme un cadeau de fête, un enluminure gravée au laser représentant le vieillard éploré tenant son chien et l’innocent enfant ; et quand Mme Verdurin, sentant qu’il lui était impossible de persuader les parents du pianiste de le laisser jouer deux heures, lui avait dit qu’il faudrait que ce tour de jardinage, s’il continue, lui ferait mal aux seins, le vieillard avait éclaté de rire.– Comment, madame Verdurin, vous


In [7]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering

model_checkpoint = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
device = torch.device("cpu")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="finetuned-proust", tokenizer=tokenizer, device=device
)

In [14]:
txt = "Ce n'est qu'après avoir écouté We are the champions de Queen"

In [16]:
print(pipe(txt, num_return_sequences=1, max_length=200)[0]["generated_text"])

Ce n'est qu'après avoir écouté We are the champions de Queen que je comprends ce que cela veut dire. Quand je dis que nous sommes les champions, je veux dire que nous sommes le plus souvent les seuls tenants du titre. Et je ne parle pas des titres de nage, des titres de saut, des titres de parapente, des titres de cirque, des titres de théâtre, qui ne sont pour moi que des titres de snobisme. Quand je dis que nous sommes les seuls tenants du titre, je ne parle pas non plus des titres de banalité, des titres de bon voisinage, des titres de fidélité, des titres de respect, qui ne sont que des titres de snobisme. Quand je dis que nous sommes les seuls tenants du titre, je ne parle pas non plus des titres de beauté, des titres de bon physique, des titres de bon moral, qui ne sont que des titres de snobisme.Quand je dis que nous sommes les seuls ten
